In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow
import time
import random

# randomOverlay(imagename)

In [ ]:
def randomOverlay(imagename):
  notempty = cv2.subtract(imagename,imagename)


  biggest_x = int(notempty.shape[0]/20)
  biggest_y = int(notempty.shape[1]/20)

  for i in range(2000):

    boxi_x=random.randrange(0,notempty.shape[0])
    boxi_dx=random.randrange(0,int(biggest_x))
    if boxi_x+boxi_dx>notempty.shape[0]:
      boxi_dx=notempty.shape[0]-boxi_x

    boxi_y=random.randrange(0,notempty.shape[1])
    boxi_dy=random.randrange(0,int(biggest_y))
    if boxi_y+boxi_dy>notempty.shape[1]:
      boxi_dy=notempty.shape[1]-boxi_y

    boxi_r=random.randrange(0,180)
    boxi_g=random.randrange(0,180)
    boxi_b=random.randrange(0,180)

    # print(boxi_x,boxi_dx,boxi_y,boxi_dy,boxi_rgb,boxi_value)

    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,0] += boxi_r
    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,1] += boxi_g
    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,2] += boxi_b

    notempty[notempty>255] = 255

  # plt.figure(figsize=(8,8))
  # plt.imshow(cv2.cvtColor(notempty, cv2.COLOR_BGR2RGB))
  # plt.axis('off')
  # plt.show()

  return notempty

# semiRandomOverlay()

In [ ]:
def semiRandomOverlay(imagename):
  notempty = cv2.subtract(imagename,imagename)


  x_or_y = int(notempty.shape[0]/35)
  y_or_x = int(notempty.shape[1]/35)

  for i in range(2000):

    xy = random.randint(0,1)
    if (xy==0):
      x = x_or_y
      y = y_or_x
    else:
      x = y_or_x
      y = x_or_y

    boxi_x=random.randrange(0,notempty.shape[0])
    boxi_dx=x
    if boxi_x+boxi_dx>notempty.shape[0]:
      boxi_dx=notempty.shape[0]-boxi_x

    boxi_y=random.randrange(0,notempty.shape[1])
    boxi_dy=y
    if boxi_y+boxi_dy>notempty.shape[1]:
      boxi_dy=notempty.shape[1]-boxi_y

    boxi_r=random.randrange(0,180)
    boxi_g=random.randrange(0,180)
    boxi_b=random.randrange(0,180)

    # print(boxi_x,boxi_dx,boxi_y,boxi_dy,boxi_rgb,boxi_value)

    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,0] += boxi_r
    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,1] += boxi_g
    notempty[boxi_x:boxi_x+boxi_dx,boxi_y:boxi_y+boxi_dy,2] += boxi_b

    notempty[notempty>255] = 255

  # plt.figure(figsize=(8,8))
  # plt.imshow(cv2.cvtColor(notempty, cv2.COLOR_BGR2RGB))
  # plt.axis('off')
  # plt.show()

  return notempty, x, y

# ObscureImage()

In [ ]:
def ObscureImage(imagepath,
                siftDrawType="default", keypointColor=None,
                rotation1=180, flip1='none', overlay1divisor=3, addWeight1=0.25,
                addRandom = True, semiRandom = False, randomDivisor=1, randomWeight=0.05,
                rotation2=180, flip2='none', overlay2divisor=3, addWeight2=0.05,
                cannythresh1=50,cannythresh2=100,
                figureSize=(15,30)):

  startTime = time.perf_counter()

  orig = cv2.imread(imagepath)

  cv2.imwrite("orig.png",orig)

  # Step 1 - Sobel XY of greyscale
  step1_0 = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
  step1 = cv2.Sobel(src=step1_0, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5)

  cv2.imwrite("step1.png",step1)
  step1 = cv2.imread("step1.png")

  # Step 2 - Find Sift Keypoints
  sift = cv2.xfeatures2d.SIFT_create()
  keypoints_sift, descriptors_sift = sift.detectAndCompute(orig, None)

  if keypointColor==None:
    if siftDrawType=="rich":
      step2 = cv2.drawKeypoints(orig, keypoints_sift, 0,  flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) #this is for illustrative purposes, not for processing
    else:
      step2 = cv2.drawKeypoints(orig, keypoints_sift, 0,  flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT) #this is for illustrative purposes, not for processing

  else:
    if siftDrawType=="rich":
      step2 = cv2.drawKeypoints(orig, keypoints_sift, 0, color=keypointColor, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) #this is for illustrative purposes, not for processing
    else:
      step2 = cv2.drawKeypoints(orig, keypoints_sift, 0, color=keypointColor, flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT) #this is for illustrative purposes, not for processing

  cv2.imwrite("step2.png",step2)
  step2 = cv2.imread("step2.png")

  # Step 3 - Create overlay: draw keypoints over sobel, rotate, decrease intensity
  if siftDrawType=="rich":
    step3_1 = cv2.drawKeypoints(step1, keypoints_sift, 0, color=keypointColor, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
  else:
    step3_1 = cv2.drawKeypoints(step1, keypoints_sift, 0, color=keypointColor, flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT)

  if rotation1==90 or rotation1==270:
    if rotation1==90:
      step3_2 = cv2.rotate(step3_1,cv2.ROTATE_90_CLOCKWISE)
    else:
      step3_2 = cv2.rotate(step3_1,cv2.ROTATE_90_COUNTERCLOCKWISE)
    step3_2 = cv2.resize(step3_2, (orig.shape[1],orig.shape[0]))
  elif rotation1==180:
    step3_2 = cv2.rotate(step3_1,cv2.ROTATE_180)
  elif rotation1==0:
    step3_2=step3_1
  else:
    print("Error: Cannot rotate by ",rotation1,"degrees; defaulting to 180")
    step3_2 = cv2.rotate(step3_1,cv2.ROTATE_180)

  if flip1=="x":
    step3_3=cv2.flip(step3_2,0)
  elif flip1=="y":
    step3_3=cv2.flip(step3_2,1)
  elif flip1=="none":
    step3_3=step3_2
  else:
    print("Error:\"",flip1,"\" is not a valid flip parameter; defaulting to no flip")
    step3_3=step3_2

  cv2.imwrite("step3_2.png",step3_2)
  step3_2 = cv2.imread("step3_2.png")

  step3 = step3_3/overlay1divisor

  cv2.imwrite("step3.png",step3)
  step3 = cv2.imread("step3.png")

  # Step 4 - Add Step3 to orig
  step4 = cv2.addWeighted(step3, addWeight1, orig, 1-addWeight1, 0.0, 0)

  cv2.imwrite("step4.png",step4)
  step4 = cv2.imread("step4.png")

  # Step 5 - Add random overlay if so desired
  if (addRandom==True):
    if (semiRandom==True):
      step5_0, semix, semiy = semiRandomOverlay(step4) #it doesn't actually matter what we plug-in here, as long as it's the same shape as the resulting image
    else:
      step5_0 = randomOverlay(step4) #it doesn't actually matter what we plug-in here, as long as it's the same shape as the resulting image
    step5_0 = step5_0/randomDivisor

    cv2.imwrite("step5_0.png",step5_0)
    step5_0 = cv2.imread("step5_0.png")

    step5 = cv2.addWeighted(step5_0, randomWeight, step4, 1-randomWeight, 0.0, 0)

    cv2.imwrite("step5.png",step5)
    step5 = cv2.imread("step5.png")

  else: # no random overlay
    step5_0 = cv2.subtract(step4,step4)
    step5_0 = cv2.putText(step5_0, 'No Random Overlay', (50,50), cv2.FONT_HERSHEY_SIMPLEX,
                          1, (255,255,255),2,cv2.LINE_AA)
    randomWeight=0
    step5= step4

 # step 6 - Find canny of Step4 & rotate
  step6 = cv2.Canny(image=step5, threshold1=cannythresh1, threshold2= cannythresh2)

  if rotation2==90 or rotation2==270:
    if rotation2==90:
      step6 = cv2.rotate(step6,cv2.ROTATE_90_CLOCKWISE)
    else:
      step6 = cv2.rotate(step6,cv2.ROTATE_90_COUNTERCLOCKWISE)
    step6 = cv2.resize(step6, (orig.shape[1],orig.shape[0]))
  elif rotation2==180:
    step6 = cv2.rotate(step6,cv2.ROTATE_180)
  elif rotation2==0:
    step6=step6
  else:
    print("Cannot rotate by ",rotation2,"degrees; defaulting to 180")
    step6 = cv2.rotate(step6,cv2.ROTATE_180)

  if flip2=="x":
    step6=cv2.flip(step6,0)
  elif flip2=="y":
    step6=cv2.flip(step6,1)
  elif flip2=="none":
    step6=step6
  else:
    print("Error:\"",flip2,"\" is not a valid flip parameter; defaulting to no flip")

  cv2.imwrite("step6_0.png",step6)
  step6_0 = cv2.imread("step6_0.png")

  step6 = step6/ overlay2divisor

  cv2.imwrite("step6.png",step6)
  step6 = cv2.imread("step6.png")

  # step 7 - Add step6 to Step4
  step7 = cv2.addWeighted(step6, addWeight2, step5, 1-addWeight2, 0.0, 0)

  cv2.imwrite("step7.png",step7)
  step7 = cv2.imread("step7.png")

  # Step 8 - Adjust brightness of step7
  #step7max = step7.max()
  #step8 = step7/step7max*255

  step8 = step7 * (1/(1-(addWeight1+addWeight2)))
  step8[step8>255] = 255

  cv2.imwrite("step8.png", step8)
  step8 = cv2.imread("step8.png")

  randomnum=str(random.randrange(0,1000000))
  finalfilepath="obscured_"+randomnum+"_"+imagepath
  cv2.imwrite(finalfilepath,step8)

  endTime = time.perf_counter()
  totalTime = endTime - startTime

  readMeFile=finalfilepath+".readme"
  with open(readMeFile,'w') as f:
    f.write("Original image: "+imagepath)
    f.write("\nFinal Image: "+finalfilepath)
    f.write(f"\n {totalTime:0.4f} seconds")
    f.write("\n")
    f.write("\nOverlay 1:")
    f.write("\n   Sobel Edge Detection")
    if keypointColor==None:
      keypointColor = "All"
    f.write("\n   Sift Keypoints, DrawType: "+siftDrawType+" Color: "+keypointColor)
    f.write("\n   Rotation: "+str(rotation1))
    f.write("\n   Flip: "+str(flip1))
    f.write("\n   Overlay intensity: 1/"+str(overlay1divisor))
    f.write("\n   Overlay weight: "+str(addWeight1))
    f.write("\n")
    f.write("\nOverlay 2:")
    if (addRandom == True):
      f.write("\n   Randomly Colored Boxes")
      if (semiRandom == True):
        f.write("\n   Boxes of same shape: "+str(semix)+"x"+str(semiy)+" pixels")
      f.write("\n   Overlay intensity: 1/"+str(randomDivisor))
      f.write("\n   Overlay weight: "+str(randomWeight))
      f.write("\n")
      f.write("\nOverlay 3:")
    f.write("\n   Canny Edge Detection")
    f.write("\n     Thresholds: "+str(cannythresh1)+", "+str(cannythresh2))
    f.write("\n   Rotation: "+str(rotation2))
    f.write("\n   Flip: "+str(flip2))
    f.write("\n   Overlay intensity: 1/"+str(overlay2divisor))
    f.write("\n   Overlay weight: "+str(addWeight2))



  print("Final printed to",finalfilepath)
  print(".readme printed to",readMeFile)
  print("Total Time: ",totalTime,"seconds")

  plt_fileName=finalfilepath+"_process.png"

  # Finally display all steps
  plt.figure(figsize=figureSize)

  plt.subplot(431)
  plt.imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Original')

  plt.subplot(432)
  plt.imshow(cv2.cvtColor(step1, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Step 1 - Sobel XY Edge Detection')

  plt.subplot(433)
  plt.imshow(cv2.cvtColor(step2, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Step 2 - Find Sift Keypoints')

  plt.subplot(434)
  plt.imshow(cv2.cvtColor(step3_2, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Step 3 - First overlay: Keypoints over Sobel')

  plt.subplot(435)
  plt.imshow(cv2.cvtColor(step4, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Step 4 - Add first overlay')

  if (addRandom==True):
    plt.subplot(436)
    plt.imshow(cv2.cvtColor(step5_0, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Step 5.0 - Create random overlay')

    plt.subplot(437)
    plt.imshow(cv2.cvtColor(step5, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Step 5 - Add random overlay')
  else:
    plt.subplot(436)
    plt.axis('off')
    plt.title( 'No random overlay')

  plt.subplot(438)
  plt.imshow(cv2.cvtColor(step6_0, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Step 6 - Final overlay: Canny Edge Detection')

  plt.subplot(439)
  plt.imshow(cv2.cvtColor(step7, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Step 7 - Add final overlay')

  plt.subplot(4,3,10)
  plt.imshow(cv2.cvtColor(step8, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Step 8 - Adjust brightness/contrast')

  plt.subplot(4,3,11)
  plt.imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Original')

  # show subtraction
  check_subtract = cv2.subtract(step8,orig)
  check_subtract = check_subtract*int(255/check_subtract.max())
  plt.subplot(4,3,12)
  plt.imshow(cv2.cvtColor(check_subtract, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title('Subtraction, Final vs. Original')

  plt.savefig(plt_fileName)
  plt.show()

# Use ObscureImage

In [ ]:
ObscureImage("Baachli_Fabian_2.png")

In [ ]:
ObscureImage("Baachli_Fabian_2.png",
             rotation1=0,flip1="x",addWeight1=0.2,
             rotation2=0,flip2="y",addWeight2=0.1)